 <h1 style="text-align: center; color: RED">AB TESTING</h1>

<h2 style="text-align: center; color: RED">İKİ GRUP ORAN KARŞILAŞTIRMA(İKİ ÖRNEKLEM ORAN TESTİ)</h2>

İki oran arasında karşılaştırma yapmak için kullanılır. Kullanılacak hipotezler:

![Test](datasets/5.png)

Oran için kullanılacak formül(örneklem sayısı 30'dan büyük olmalıdır.):

![Test](datasets/6.png)

In [28]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multicomp import MultiComparison

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [9]:
############################
# Uygulama : Yeni Tasarımın Dönüşüm Oranı ile Eski Tasarımın Dönüşüm Oranı Arasında İstatistiksel Olarak Fark Var Mı?
############################
#H0: p1=p2(Oranlar arasında fark yoktur.)
#H1: p1!=p2(fark vardır)

In [13]:
basari_sayisi = np.array([300, 250])
gozlem_sayisi = np.array([1000, 1100])
proportions_ztest(count=basari_sayisi, nobs=gozlem_sayisi)

(3.7857863233209255, 0.0001532232957772221)

p < 0.05 olduğu için H0 reddedilir yani oranlar arasında fark vardır.

In [20]:
basari_sayisi / gozlem_sayisi #ilk gözlem oran olarak daha başarılı

array([0.3       , 0.22727273])

In [22]:
############################
# Uygulama : Kadın ve Erkeklerin Hayatta Kalma Oranları Arasında İstatistiksel Olarak Fark Var Mı?
############################
#H0: p1=p2(Oranlar arasında fark yoktur.)
#H1: p1!=p2(fark vardır)

In [24]:
df = sns.load_dataset('titanic')

In [26]:
df.loc[df['sex']=='female','survived'].mean()

0.7420382165605095

In [28]:
df.loc[df['sex']=='male','survived'].mean()

0.18890814558058924

In [34]:
female_succ_count = df.loc[df['sex']=='female','survived'].sum()
male_succ_count = df.loc[df['sex']=='male','survived'].sum()

In [38]:
test_stat, pvalue = proportions_ztest(count=[female_succ_count,male_succ_count], 
                                      nobs=[df.loc[df['sex']=='female','survived'].shape[0], df.loc[df['sex']=='male','survived'].shape[0]])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 16.2188, p-value = 0.0000


p < 0.05 olduğundan H0 reddedilir. Yani oransal olarak bir fark vardır.

<h2 style="text-align: center; color: RED">İKİDEN FAZLA GRUP ORTALAMASI KARŞILAŞTIRMA(ANOVA)</h2>


![Test](datasets/7.png)

![Test](datasets/8.png)

In [1]:
############################
# Uygulama : Haftanın Günleri Arasında Ödenen Hesaplar Aarsındaİstatistiksel Olarak Fark Var Mı?
############################


In [7]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99000,1.01000,Female,No,Sun,Dinner,2
1,10.34000,1.66000,Male,No,Sun,Dinner,3
2,21.01000,3.50000,Male,No,Sun,Dinner,3
3,23.68000,3.31000,Male,No,Sun,Dinner,2
4,24.59000,3.61000,Female,No,Sun,Dinner,4


In [9]:
df.groupby('day')['total_bill'].mean()

C:\Users\ASLI\AppData\Local\Temp\ipykernel_14828\1881343769.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('day')['total_bill'].mean()


day
Thur   17.68274
Fri    17.15158
Sat    20.44138
Sun    21.41000
Name: total_bill, dtype: float64

In [ ]:
#1)HİPOTEZLERİ KUR
#H0: m1=m2=m3=M(Ortalamalar arasında fark yoktur.)
#H1: en az biri farklıdır

In [11]:
#2)VARSAYIM KONTROLU
#Normallik varsayımı
#Varyans Homojenliği
#Varsayımlar sağlanırsa one way anova testi, sağlanmıyorsa kruskal testi yapılır.

In [15]:
#Normallik Varsayım
#H0 varsayım sağlanmaktadır.
#H1 sağlanmamaktadır.
for group in list(df['day'].unique()):
    pvalue = shapiro(df.loc[df['day']==group,'total_bill'])[1]
    print(group, 'p-value %.4f' % pvalue)


Sun p-value 0.0036
Sat p-value 0.0000
Thur p-value 0.0000
Fri p-value 0.0409


p value değerleri 0.05'den küçük olduğu için H0 reddedilir. Yani normallik varsayımı sağlanmamaktadır.

In [22]:
#Varyans Homojenliği
#H0 varsayım sağlanmaktadır.
#H1 sağlanmamaktadır.
test_stat, pvalue = levene(df.loc[df['day']=='Sun','total_bill'],
                           df.loc[df['day']=='Sat','total_bill'],
                           df.loc[df['day']=='Thur','total_bill'],
                           df.loc[df['day']=='Fri','total_bill'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.6654, p-value = 0.5741


p value değeri 0.05'ten büyük oldugu için H0 reddedilemez yani varyans homojenliği sağlanmaktadır. Fakat normallik varsayımı sağlanmadığı için non parametrik tets yapılacaktır.

In [25]:
#3)HİPOTEZ TESTİ
test_stat, pvalue = kruskal(df.loc[df['day']=='Sun','total_bill'],
                           df.loc[df['day']=='Sat','total_bill'],
                           df.loc[df['day']=='Thur','total_bill'],
                           df.loc[df['day']=='Fri','total_bill'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 10.4031, p-value = 0.0154


p değeri 0.05'den küçük olduğu için H0 reddedilir. Yani gruplar arasında istatistiksel olarak bir fark vardır.

In [30]:
#Farklılık hangi gruptan kaynaklanıyor.Bunu ikili karşılaştırmalar ile bulmaya çalışacaz
comparison = MultiComparison(df['total_bill'],df['day'])
tukey = comparison.tukeyhsd(0.05)
print(tukey)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
   Fri    Sat   3.2898 0.4541 -2.4799  9.0595  False
   Fri    Sun   4.2584 0.2371 -1.5856 10.1025  False
   Fri   Thur   0.5312 0.9957 -5.4434  6.5057  False
   Sat    Sun   0.9686 0.8968 -2.6088   4.546  False
   Sat   Thur  -2.7586 0.2374 -6.5455  1.0282  False
   Sun   Thur  -3.7273 0.0668 -7.6264  0.1719  False
----------------------------------------------------


İkili karşılaştırmalara baktığımızda p valu değerleri 0.05'den büyük oldugu için H0 reddedilemez. Ynai Farklılık yoktur diyoruz.Ama grup bazında baktığımızda bir fark vardı.